# Problema2 - REALIZAR SOBRE ARCHIVO .PY  (NO JUPYTER) 


Lea el archivo 'winemag-data-130k-v2.csv' y realice lo siguiente:

- Explore el dataframe según lo visto en clase
- Realice al menos  4 renombre de columna y 3 creación de nuevas columnas según criterio. Deberá crear las columnas que crea conveniente. 
    Ejemplo: Según el país etiquetelos según continente.

- Genere 4 reportes distintos(podria agrupando, filtrar, ordenar, etc). Deberá elegir que reportes realizar
    - Ejemplo: Por contienente mostrar los vinos mejor puntuados
    - Ejemplo2: Promedio de precio de vino y cantidad de reviews obtenidos según pais. Ordenado de mejor a peor.



- Exporte los 4 reportes a 4 formatos de archivos diversos, csv, excel, sqllite, mongodb...
   
   Para guardar datos de datos agrupados, puede revisar esta solución https://stackoverflow.com/questions/25789264/pandas-dataframegroupby-export-to-excel

- Envie uno de los reportes generados por correo, mostrar un pantallazo del correo enviado




### Datos 

winemag-data-130k-v2.csv
- https://github.com/gdelgador/ProgramacionPython202401/blob/main/Modulo5/src/winemag-data-130k-v2.csv

País según continente:
- https://gist.githubusercontent.com/kintero/7d1db891401f56256c79/raw/a61f6d0dda82c3f04d2e6e76c3870552ef6cf0c6/paises.csv
 






In [ ]:
import pandas as pd
import sqlite3
from pymongo import MongoClient

# Cargar CSV
df = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

# ====== Exploración básica ======
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

# ====== Renombrar columnas ======
df = df.rename(columns={
    'country': 'pais',
    'description': 'descripcion',
    'points': 'puntos',
    'price': 'precio',
    'variety': 'variedad'
})

# ====== Crear nuevas columnas ======
# 1. Continente por país
def obtener_continente(pais):
    if pais in ['France', 'Italy', 'Spain', 'Germany', 'Portugal']:
        return 'Europa'
    elif pais in ['US', 'Canada']:
        return 'América del Norte'
    elif pais in ['Argentina', 'Chile']:
        return 'América del Sur'
    elif pais in ['Australia', 'New Zealand']:
        return 'Oceanía'
    elif pais in ['South Africa']:
        return 'África'
    else:
        return 'Otro'

df['continente'] = df['pais'].apply(obtener_continente)

# 2. Rango de precios
df['rango_precio'] = pd.cut(df['precio'], bins=[0, 15, 30, 50, 100, 1000], 
                            labels=['Económico', 'Accesible', 'Medio', 'Premium', 'Lujoso'])

# 3. Longitud de descripción
df['long_desc'] = df['descripcion'].apply(lambda x: len(str(x)))

# ====== Reportes ======

# Reporte 1: Vinos mejor puntuados por continente
reporte1 = df.loc[df.groupby('continente')['puntos'].idxmax()][['pais', 'continente', 'variedad', 'puntos', 'precio']]

# Reporte 2: Promedio de precio y cantidad de reviews por país
reporte2 = df.groupby('pais').agg(
    promedio_precio=('precio', 'mean'),
    cantidad_reviews=('descripcion', 'count')
).sort_values(by='promedio_precio', ascending=False)

# Reporte 3: Variedades más comunes por país (Top 5)
reporte3 = df.groupby(['pais', 'variedad']).size().reset_index(name='cantidad')
reporte3 = reporte3.sort_values(by='cantidad', ascending=False).groupby('pais').head(5)

# Reporte 4: Puntuación promedio por rango de precio
reporte4 = df.groupby('rango_precio')['puntos'].mean().reset_index().sort_values(by='puntos', ascending=False)

# ====== Exportar archivos ======

# 1. CSV
reporte1.to_csv("reporte1_vinos_puntuados_continente.csv", index=False)

# 2. Excel
reporte2.to_excel("reporte2_precio_reviews_pais.xlsx")

# 3. SQLite
conn = sqlite3.connect('vinos.db')
reporte3.to_sql('variedades_por_pais', conn, if_exists='replace', index=False)
conn.close()

# 4. MongoDB
try:
    client = MongoClient('mongodb://localhost:27017/')
    db = client['vino_db']
    collection = db['puntuacion_por_rango_precio']
    collection.drop()  # Limpia colección anterior
    collection.insert_many(reporte4.to_dict('records'))
    print("Reporte 4 guardado en MongoDB")
except Exception as e:
    print("No se pudo conectar a MongoDB:", e)

# FIN
print("Todos los reportes han sido generados y exportados.")


pip install pandas openpyxl pymongo

